In [195]:
import pandas as pd 
import chardet

In [196]:
# Miramos que codificacion tienen los archivos y pasamos el resultados a df con ayuda
# de pandas para ayudarnos mejor
with open("src/ventas(in).csv", "rb") as f:
    result = chardet.detect(f.read())
df_ventas = pd.read_csv("src/ventas(in).csv",sep=';',encoding=result["encoding"], parse_dates=["Fecha de venta"], dayfirst=True)
print("Codificacion de ventas -> "+result["encoding"])

with open("src/vendedores(in).csv", "rb") as f:
    result = chardet.detect(f.read())
df_vendedores = pd.read_csv("src/vendedores(in).csv",sep=';',encoding=result["encoding"])
print("Codificacion de vendedores -> "+result["encoding"])

with open("src/paises(in).csv", "rb") as f:
    result = chardet.detect(f.read())
df_paises = pd.read_csv("src/paises(in).csv", sep=',',encoding=result["encoding"])
print("Codificacion de paises -> "+result["encoding"])

Codificacion de ventas -> ISO-8859-1
Codificacion de vendedores -> ISO-8859-1
Codificacion de paises -> utf-8


In [197]:

#Creamos 2 DataFrames, donde buscamos y separamos por valores de df_paises
#Luego, volvemos a juntarlo en uno solo
df_semi_1 = df_ventas.merge(df_paises,left_on='Región',right_on='ISO2', how='left')
df_semi_1 = df_semi_1.dropna()

df_semi_2 = df_ventas.merge(df_paises,left_on='Región',right_on='País', how='left')
df_semi_2 = df_semi_2.dropna()

df_ventas = df_semi_1.merge(df_semi_2, how='outer')

df_ventas.drop(columns=["Región","ISO2"], inplace=True)
df_ventas.rename(columns={"País":"Región"}, inplace=True)

print(df_ventas)

     Número de Pedido Fecha de venta                                Producto  \
0              104615     2024-03-17               innovate killer e-markets   
1              104844     2024-05-15       generate holistic functionalities   
2              105001     2024-02-01      aggregate compelling web-readiness   
3              105008     2024-02-29              embrace impactful networks   
4              105054     2024-04-19          architect compelling e-tailers   
..                ...            ...                                     ...   
469            990322     2024-01-09        utilize holistic infrastructures   
470            991362     2024-06-02       transition seamless architectures   
471            992366     2024-04-23         synthesize virtual applications   
472            996046     2024-01-07               e-enable robust solutions   
473            997304     2024-04-04  aggregate collaborative info-mediaries   

    Operador  Cantidad vendida  Ingreso

In [198]:
#Para saber los idiomas (EN,ES) en distintos paises y de personas
#Lo necesitaremos para después
df_equipos = df_vendedores[df_vendedores["Status"] != "baja"]
df_equipos = df_equipos.groupby(["Idioma","País","Equipo","Status"]).size().reset_index(name="E")
df_equipos.rename(columns={"País":"Región"}, inplace=True)

print(df_equipos)



   Idioma          Región                Equipo  Status  E
0      EN        Alemania          VC-España-EN  activo  1
1      EN        Alemania  VC-Estados Unidos-EN  activo  1
2      EN          Brasil        VC-Alemania-ES  activo  1
3      EN           China          VC-España-EN  activo  3
4      EN  Estados Unidos        VC-Alemania-EN  activo  1
5      EN  Estados Unidos          VC-España-EN  activo  1
6      EN  Estados Unidos  VC-Estados Unidos-EN  activo  1
7      EN         Francia        VC-Alemania-ES  activo  1
8      EN         Francia  VC-Estados Unidos-EN  activo  2
9      EN         Francia  VC-Estados Unidos-ES  activo  1
10     EN          Italia  VC-Estados Unidos-EN  activo  1
11     EN           Japón          VC-España-EN  activo  1
12     ES        Alemania        VC-Alemania-EN  activo  1
13     ES        Alemania          VC-España-ES  activo  1
14     ES           China        VC-Alemania-EN  activo  1
15     ES          España        VC-Alemania-ES  activo 

In [199]:
# Tras obtener un DF de vendedores que podamos usar,
# Ahora toca crear DF para ventas diarias y acumuladas
# mensuales

df_ventas["Día"] = df_ventas["Fecha de venta"].dt.strftime("%Y-%m-%d")
df_ventas["Mensual"] = df_ventas["Fecha de venta"].dt.strftime("%Y-%m")

# Realizamos merge de DF ventas y vendedores
df_ventas_final = df_ventas.merge(df_equipos[["Región", "Idioma", "Equipo"]], on="Región", how="left")

#Guardamos un df con las ventas diarias
df_ventas_diarias = df_ventas_final.groupby(["Día", "Región", "Idioma","Equipo","Tipo de venta"]).agg({"Cantidad vendida": "sum"}).reset_index()
print(df_ventas_diarias)
#Guardamos un df con las ventas por mes
df_ventas_mensuales = df_ventas_final.groupby(["Mensual", "Región", "Idioma","Equipo","Tipo de venta"]).agg({"Cantidad vendida": "sum"}).reset_index()
print(df_ventas_mensuales)

             Día   Región Idioma                Equipo Tipo de venta  \
0     2024-01-01   Brasil     EN        VC-Alemania-ES      caliente   
1     2024-01-01    China     EN          VC-España-EN          fría   
2     2024-01-01    China     ES        VC-Alemania-EN          fría   
3     2024-01-01  Francia     EN        VC-Alemania-ES      caliente   
4     2024-01-01  Francia     EN  VC-Estados Unidos-EN      caliente   
...          ...      ...    ...                   ...           ...   
1165  2024-07-03  Francia     ES        VC-Alemania-EN      caliente   
1166  2024-07-03  Francia     ES          VC-España-ES      caliente   
1167  2024-07-03  Francia     ES  VC-Estados Unidos-ES      caliente   
1168  2024-07-03    Japón     EN          VC-España-EN          fría   
1169  2024-07-03    Japón     ES  VC-Estados Unidos-ES          fría   

      Cantidad vendida  
0                    8  
1                    6  
2                    6  
3                    2  
4         

In [200]:
# Finalmente, generamos los reportes en Excel
with pd.ExcelWriter("Reportes_Ventas.xlsx") as writer:
    df_ventas_diarias.to_excel(writer,sheet_name="Ventas diarias",index=False)
    df_ventas_mensuales.to_excel(writer,sheet_name="Ventas acumuladas",index=False)
    
print("Reporte generado con éxito")

Reporte generado con éxito
